## Chatbot Logic

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
!pip install -qqq transformers datasets

In [3]:
import os
import torch
import random
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from IPython.display import Markdown, display
from datasets import Dataset

In [4]:
try:
    os.chdir("/content/drive/MyDrive/skripsi")
    print("Directory changed")
except OSError:
    print("Error: Can't change the Current Working Directory")

Directory changed


In [5]:
# Constants
start_message = "==== Hello! I am Alex and I am your virtual friend. If you need a listening ear, I'm always here. To end the chat, input 'exit' in the chatbox. ===="

prevention_messages = ["Are you okay? How long have you been feeling this way?",
                       "That sounds so painful, and I appreciate you sharing that with me. How can I help?",
                       "I know things seem bleak now, but it can be hard to see possible solutions when you feel so overwhelmed.",
                       "I'm concerned about you because I care, and I want to offer support however I can. You can talk to me.",
                       "I'm always here if you feel like talking.",
                       "I'm always here to listen, but do you think a therapist could help a little more?",
                       "Have you thought about talking to a therapist?",
                       "You can withstand any storm and when you are too tired to stand, I will hold you up. You are never alone.",
                       "You know I’m always here for you.",
                       "You’re allowed to have bad days, but remember tomorrow is a brand new day.",
                       "You’ve got a place here on Earth for a reason.",
                       "It's okay to have such thoughts but if they become overwhelming, don't keep it to yourself. I am here for you.",
                       "Everything is a season, and right now you’re in winter. It’s dark and cold and you can’t find shelter, but one day it’ll be summer, and you’ll look back and be grateful you stuck it out through winter.",
                       "I know you are going through a lot and you’re scared, but you will never lose me.",
                       "I know it feels like a lot right now. It’s OK to feel that way.",
                       "Is there anything I can do to make this day go easier for you?"]

helpline_message = "In times of severe distress where you need to speak with someone immediately, these are suicide hotline services available for you. You will be speaking with volunteers or professionals who are trained to deal with suicide crisis. Samaritans of Singapore (SOS; 24 hours): 1800 221 4444 Mental Health Helpline (24 hours): 6389 2222 Singapore Association for Mental Health (SAMH) Helpline: 1800 283 7019"

### Functions

In [6]:
def printmd(string):
    display(Markdown(string))

In [7]:
# Untuk load tokenizer dan model generate response
def load_tokenizer_and_model(model="microsoft/DialoGPT-large"):
  tokenizer = AutoTokenizer.from_pretrained(model)
  model = AutoModelForCausalLM.from_pretrained(model)

  return tokenizer, model

In [8]:
# Untuk load tokenizer dan model sentiment analysis (detect suicidal intention)
def load_suicide_tokenizer_and_model(tokenizer="bert-base-uncased", model='./Models/bert/'):
  suicide_tokenizer = AutoTokenizer.from_pretrained(tokenizer)
  suicide_model = AutoModelForSequenceClassification.from_pretrained('./Models/bert/')

  return suicide_tokenizer, suicide_model

In [9]:
# Detecting Suicidal Intention
def check_intent(text):
  global suicide_tokenizer, suicide_model
  tokenised_text = suicide_tokenizer.encode_plus(text, return_tensors="pt")
  logits = suicide_model(**tokenised_text)[0]
  prediction = round(torch.softmax(logits, dim=1).tolist()[0][1])
  return prediction

In [10]:
# Generate Response
# Requires Editing
def generate_response(tokenizer, model, chat_round, chat_history_ids):
  user_input = input(">> You: ")

  if user_input == "exit":
    raise Exception("End of Conversation")

  # Encode user input and End-of-String (EOS) token
  new_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors='pt')

  # Append tokens to chat history
  bot_input_ids = torch.cat([chat_history_ids, new_input_ids], dim = -1) if chat_round > 0 else new_input_ids

  # Generate response given maximum chat length history of 1250 tokens
  chat_history_ids = model.generate(bot_input_ids, max_length=1250, pad_token_id=tokenizer.eos_token_id)

  # Print response based on intent
  if check_intent(user_input):
    printmd("*Alex:* {}".format(random.choice(prevention_messages)))
    printmd("{}".format(helpline_message))
  else:
    printmd("*Alex:* {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))

  # Return the chat history ids
  return chat_history_ids

In [11]:
# Initialize chatbot tokenizer and model
tokenizer, model = load_tokenizer_and_model()

# Initialize chatbot history variable
chat_history_ids = None

# Initialise suicide detection tokenizer and model
suicide_tokenizer, suicide_model = load_suicide_tokenizer_and_model()

In [12]:
# Edit sebagaimana mungkin...
def start_chatbot(n=1000):
  global tokenizer, model, chat_history_ids

  print(start_message)

  try:
    for chat_round in range(n):
      chat_history_ids = generate_response(tokenizer, model, chat_round, chat_history_ids)
  except Exception as e:
    printmd("*Alex:* See ya")


## Serverside (FastAPI)



In [13]:
!pip install fastapi==0.103.2 nest-asyncio pyngrok uvicorn

In [14]:
messages = ["Are you okay? How long have you been feeling this way?",
                       "That sounds so painful, and I appreciate you sharing that with me. How can I help?",
                       "I know things seem bleak now, but it can be hard to see possible solutions when you feel so overwhelmed.",
                       "I'm concerned about you because I care, and I want to offer support however I can. You can talk to me.",
                       "I'm always here if you feel like talking.",
                       "I'm always here to listen, but do you think a therapist could help a little more?",
                       "Have you thought about talking to a therapist?",
                       "You can withstand any storm and when you are too tired to stand, I will hold you up. You are never alone.",
                       "You know I’m always here for you.",
                       "You’re allowed to have bad days, but remember tomorrow is a brand new day.",
                       "You’ve got a place here on Earth for a reason.",
                       "It's okay to have such thoughts but if they become overwhelming, don't keep it to yourself. I am here for you.",
                       "Everything is a season, and right now you’re in winter. It’s dark and cold and you can’t find shelter, but one day it’ll be summer, and you’ll look back and be grateful you stuck it out through winter.",
                       "I know you are going through a lot and you’re scared, but you will never lose me.",
                       "I know it feels like a lot right now. It’s OK to feel that way.",
                       "Is there anything I can do to make this day go easier for you?"]



In [15]:
!pip install -q -U google-generativeai

In [16]:
import pathlib
import textwrap

import google.generativeai as genai
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])
chat

In [17]:
!ngrok config add-authtoken $NGROK_API_KEY

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [18]:
from fastapi import FastAPI, Request, Body
from typing import Optional
from fastapi.middleware.cors import CORSMiddleware
import random
import nest_asyncio
from pyngrok import ngrok
import uvicorn

from pydantic import BaseModel

class Message(BaseModel):
  userMessage: str
  userId: int

class Response():
  message: str = ""
  sentiment: bool = 0

request_object = { "userMessage": "this is the message", "userId": 69 }
test = Body(..., example=request_object, description="The message content", embed=True)

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

@app.post('/chat/send/')
async def send_chat(
    message: Message = test
):
    response_object = Response()
    bot_message = chat.send_message(message.userMessage)
    check_result = check_intent(message.userMessage)

    response_object.sentiment = check_result
    if (check_result):
      response_object.message = random.choice(messages)
    else:
      response_object.message = bot_message.text

    response = {
        "message": response_object.message,
        "sentiment": response_object.sentiment
    }

    return response


# Ngrok
from google.colab import userdata
NGROK_API_KEY = userdata.get('NGROK_API_KEY')

ngrok.Client(NGROK_API_KEY)
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [55376]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://b810-35-223-115-199.ngrok-free.app
INFO:     2001:448a:7030:a41c:e7e3:d6f4:9f6f:6360:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2001:448a:7030:a41c:e7e3:d6f4:9f6f:6360:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2001:448a:7030:a41c:e7e3:d6f4:9f6f:6360:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2001:448a:7030:a41c:e7e3:d6f4:9f6f:6360:0 - "POST /chat/send/ HTTP/1.1" 200 OK
INFO:     2001:448a:7030:a41c:e7e3:d6f4:9f6f:6360:0 - "POST /chat/send/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [55376]


In [26]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://b2c1-35-223-115-199.ngrok-free.app


INFO:     Started server process [216]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2001:448a:7030:a41c:e7e3:d6f4:9f6f:6360:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2001:448a:7030:a41c:e7e3:d6f4:9f6f:6360:0 - "GET /openapi.json HTTP/1.1" 200 OK
string
INFO:     2001:448a:7030:a41c:e7e3:d6f4:9f6f:6360:0 - "POST /chat/send/ HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [216]
